In [1]:
# %load_ext autoreload
# %autoreload 2

# Update Notes: 

### 12-20-2021

* change Date -> Data_Date

### 12-21-2021

* add Data_HourMinute column

## TODO
needs to change the FY related number to dynamic number

# Tip for quick search

* Needs attention: the place where needs update or better logic
* question to be answered: the place where things are still not clear
* Manual Check: Unit test where you can drill in to find the data that leads to the check results for a specific project and specific check
* TODO: things needs to be done
* bookmark: stop point from last visit


# Admin Notes:


1. The AMTool dataset is archived daily as csv files and used for the project book check. 
The csv files are located at: 
r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'

2. The excel input files are checked daily and archived with datestamp whenever it is modified.
The continuously updated excel input files are located at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\excel'
The excel input file are archived at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\Data_MiscInput'
To recover the archived excel file used in project book check for a target date, select the excel file with latest datestamp but is still earlier than the target date.

3. The check summary export action is logged daily. It can be used for daily monitoring. 
The file export log is located at: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal\log

4. The published data are at:

    * csv files for district asset manager: http://svgcshopp.dot.ca.gov/DataLake/ProjectBookCheck/
    * csv files for HQ AM: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal
    * tableau workbook with live data source: https://tableau.dot.ca.gov/#/site/AssetManagement/workbooks/1815/views


<a id='TableOfContents'></a>

# Table Of Contents

## Data Preprocessing

### [Global Constants](#GlobalConstants)


### [Load and cleanup source data](#Read_Data)

* [Counties](#Counties)
* [Programming_Summary](#Programming_Summary)
* [ProgrammingList](#ProgrammingList)
* [SHOPP_Raw_Data](#SHOPP_Raw_Data)
* [TenYrShopp_Perf_RawData](#TenYrShopp_Perf_RawData)


## Add fields to SHOPP raw data (calculate and join)
* [Calculated Fields](#AddDataColumns)
* [Join Tables](#DataJoining)


## [Export Data](#Export_Data)



## [Final Clean Up](#FinalCleanUp)


# Import common modules

In [2]:

from datetime import datetime
import os.path

# import requests
import pandas as pd

import numpy as np
import re

import shutil

In [3]:
from itertools import product

In [4]:
import time
start_time = time.time()

In [5]:
#show dataframe without skip column
pd.options.display.max_columns = 100

In [6]:
# from config_datasource import *
import projectbookcheck_utilityfunction as uf
from constants import *

You are using the Extract API 2.0, please save the output as .hyper format


# Data clean process

* funding amount: remove dollar sign, 
* fill missing value, string, numerical, 
* remove leading single quote for string value
* strip off leading and trailing space 

* regulate column names




<a id='GlobalConstants'></a>
## Global Constants

In [7]:
filename = 'TenYrShopp_RawData_'
path_to_file = r'{}\{}.csv'.format(DATALAKE_HTTPSERVER_FOLDER, filename)
t = os.path.getmtime(path_to_file)

# File_TimeStamp = datetime.fromtimestamp(t).strftime("%m-%d-%Y_%H-%M")
Data_TimeStamp = datetime.fromtimestamp(t).strftime("%m-%d-%Y %H:%M:%S")

TARGETDATE = datetime.fromtimestamp(t).strftime("%m-%d-%Y")

In [8]:
CURRENT_FY = uf.fiscalyear (datetime.today())

In [9]:
file_export_log = open(LOG_FILE, "a")  # append mode
file_export_log.write("#####SHOPP Manager Review:{} \n".format(Data_TimeStamp))
file_export_log.close()

<a id='Read_Data'></a>

# Read Data


In [10]:
File_TimeStamp = ''

if DATA_SOURCE_TYPE == 'csv':

    filename = 'TenYrShopp_PerfM_Raw_Data_'
    df_perf_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Rawdata_Bridge_Worksheet_'
    df_brg_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), skiprows = [0], header = 0)

    filename = 'Rawdata_Drainage_Worksheet_'
    df_drain_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), 
                                    header = 0, 
#                                     keep_default_na=False,
#                                     na_values = ['',' ']
                                   )

    filename = 'Rawdata_TMS_Worksheet_'
    df_tms_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)

    filename = 'Rawdata_FishPassage_Worksheet_'
    df_fp_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)
    
    filename = 'projectbook_draft'
    filepath_draft_projectbook = r'{}\{}.csv'.format(PROJECTBOOKCHECK_HTTPSERVER_FOLDER, filename)
    df_draft_pb = pd.read_csv(filepath_draft_projectbook, header = 0)
    
    
else:
    print('skip getting csv data.')

C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
filename = 'Latest_approved_book.xlsx'

sht = 'Statewide'

df_approved_projectbook = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), sheet_name = sht) 


In [12]:
filename = 'Latest_approved_performance.xlsx'

sht = 'Statewide_various_dates'

df_approved_performance = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), sheet_name = sht) 


In [13]:
dict_rename = {
    'SHOPP ID': 'AMT_ID',
    'EFIS ID':'EFIS',
    'Tool Section': 'Latest Certified Book Tool Section',
    'Project Cost ($K)': 'Latest Certified Book Project Cost ($K)',
    'Advertised Year': 'Latest Certified Book Advertised Year',
    'Planning or Post planning? ': 'Latest Certified Book Planning or Post-Planning'
                               }
df_approved_projectbook.rename(dict_rename, axis = 1, inplace = True)

In [14]:
# df_approved_projectbook.columns

In [15]:
# df_approved_performance

## Approved project book

In [16]:
df_approved_pb_combined = pd.merge(df_approved_projectbook, 
    df_draft_pb[['AMT_ID', 'Planning or Post-Planning']], 
    how='left', 
    left_on= 'AMT_ID', right_on= 'AMT_ID'
)

In [17]:
df_approved_pb_combined['In Current Draft Book?'] = df_approved_pb_combined['Planning or Post-Planning'].apply(
lambda x: 'No' if pd.isna(x) else 'Yes'
)

In [18]:
df_approved_pb_combined['Current Draft Book Status'] = df_approved_pb_combined['Planning or Post-Planning'].apply(
lambda x: 'Not in Current Draft Book' if pd.isna(x) else x
)

In [19]:
ck_col = 'Different Project Status?'

def ck_project_status(df):
    if df['Current Draft Book Status'] != df['Planning or Post-Planning']:
        'Yes'
    else:
        return 'No'

df_approved_pb_combined[ck_col] = df_approved_pb_combined.apply(ck_project_status, axis = 1)

In [20]:
def cum_project_deleted(series):
    deleted = 0
    for v in series:
        if v == 'No':
            deleted += 1
    return deleted


In [21]:
df_approved_pb_summary = df_approved_pb_combined.groupby(['District','Latest Certified Book Planning or Post-Planning' ]).agg(
    {'AMT_ID': 'count',
    'In Current Draft Book?':cum_project_deleted,
    'Latest Certified Book Project Cost ($K)':sum,
    }
).reset_index()


In [22]:
dict_rename = {
            'Latest Certified Book Planning or Post-Planning':'Project Status',
            'Total Project Cost ($K)_sum':'Current Draft Book Total Project Cost ($K)',
            'AMT_ID': 'Latest Certified Book Number of projects',
            'In Current Draft Book?':'Number of projects Deleted',
            'Latest Certified Book Project Cost ($K)':'Latest Certified Book Total Project Cost ($K)',
              }

df_approved_pb_summary.rename(dict_rename, axis = 1, inplace=True)

In [23]:
df_approved_pb_summary['Data_TimeStamp'] = Data_TimeStamp
uf.export_hyper(df_approved_pb_summary, 'Approved_ProjectBook_Changes', LOG_FILE)

processing table: 24it [00:00, 12009.46it/s]

Table 'Extract' does not exist in extract Approved_ProjectBook_Changes.hyper, creating.


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Approved_ProjectBook_Changes.hyper to Sandbox_ProjectBookCheck_Automation...


## draft project book

In [24]:
df_draft_pb_combined = pd.merge(
    df_draft_pb, df_approved_projectbook[['AMT_ID','Latest Certified Book Tool Section', 
                                          'Latest Certified Book Project Cost ($K)',
                                          'Latest Certified Book Advertised Year']], 
    how='left', 
    left_on= 'AMT_ID', right_on= 'AMT_ID'
)

In [25]:
for c in ['Latest Certified Book Tool Section', 
          'Latest Certified Book Project Cost ($K)',
          'Latest Certified Book Advertised Year']:
    df_draft_pb_combined[c].fillna('Added to Project Book', inplace = True)

In [26]:
def ck_match_data_in_cols(df, col1, col2):
    
    if df[col2] == 'Added to Project Book':
        return df[col2]
    elif df[col1] == df[col2]:
        return 'No'
    else:
        return 'Yes'

In [27]:
ck_col = 'Any Change in RTL?'

df_draft_pb_combined[ck_col] = df_draft_pb_combined.apply(ck_match_data_in_cols, args = ['Advertised Year','Latest Certified Book Advertised Year'], axis = 1)

In [28]:
ck_col = 'Any Change in Project Cost?'

df_draft_pb_combined[ck_col] = df_draft_pb_combined.apply(ck_match_data_in_cols, args = ['Total Project Cost ($K)','Latest Certified Book Project Cost ($K)'], axis = 1)

In [29]:
ck_col = 'Any Change in Tool Section?'

df_draft_pb_combined[ck_col] = df_draft_pb_combined.apply(ck_match_data_in_cols, args = ['Section','Latest Certified Book Tool Section'], axis = 1)

In [30]:
ck_col = 'Changes in Schedule, Cost, Performance, or Tool Section?'

def combine_changes(df):
    if (df['Any Change in RTL?'] == 'Yes' 
        or df['Any Change in Project Cost?'] == 'Yes' 
        or df['Any Change in Tool Section?'] == 'Yes'):
        return 'Yes'
    else:
        return 'No'

df_draft_pb_combined[ck_col] = df_draft_pb_combined.apply(combine_changes, axis = 1)

In [31]:
ck_col = 'Changes in Schedule, Cost, Performance, or Tool Section?'

def combine_changes(df):
    if (df['Any Change in RTL?'] == 'Yes' 
        or df['Any Change in Project Cost?'] == 'Yes' 
        or df['Any Change in Tool Section?'] == 'Yes'):
        return 'Yes'
    else:
        return 'No'

df_draft_pb_combined[ck_col] = df_draft_pb_combined.apply(combine_changes, axis = 1)

In [32]:
#questions to be answered: How to calculate performance changed column
df_draft_pb_combined['Any Change in Performance (SHSMP Objectives)?'] = 'TODO'

In [33]:
df_draft_pb_combined['Data_TimeStamp'] = Data_TimeStamp
uf.export_hyper(df_draft_pb_combined, 'Draft_ProjectBook_Changes', LOG_FILE)

processing table: 782it [00:00, 7819.78it/s]

Table 'Extract' does not exist in extract Draft_ProjectBook_Changes.hyper, creating.


processing table: 1937it [00:00, 7873.55it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Draft_ProjectBook_Changes.hyper to Sandbox_ProjectBookCheck_Automation...


## Data Summary Table

In [34]:
def cum_changed(series):
    cum = 0
    changed = 0
    for v in series:
        if v == 'Yes':
            changed += 1
        cum += 1
#     print(changed, cum)
    return changed


In [35]:
def cum_project_added(series):
    added = 0
    for v in series:
        if v == 'Added to Project Book':
            added += 1
    return added


In [36]:
# df_draft_pb_summary = df_draft_pb_combined.groupby(['District', 'Planning or Post-Planning']).agg({
#     'Total Project Cost ($K)': {'Current Draft Book Total Project Cost ($K)':'sum'},
#     'AMT_ID': {'Current Draft Book Number of projects':'count'},
# #     'Any Change in RTL?': {'% with RTL change': cum_change_percent, 'Number of projects Added':cum_project_added},
# #     'Any Change in Project Cost?': {'% with Project Cost Change':cum_change_percent},
# }).reset_index()

In [37]:
df_draft_pb_summary = df_draft_pb_combined.groupby(['District', 'Planning or Post-Planning']).agg({
    'Total Project Cost ($K)': sum,
    'AMT_ID': 'count',
    'Any Change in RTL?':  [cum_changed, cum_project_added],
    'Any Change in Project Cost?':cum_changed,
}).reset_index()

df_draft_pb_summary.columns = ['_'.join(col) for col in df_draft_pb_summary.columns.values]

dict_rename = {'District_':'District',
            'Planning or Post-Planning_':'Project Status',
            'Total Project Cost ($K)_sum':'Current Draft Book Total Project Cost ($K)',
            'AMT_ID_count': 'Current Draft Book Number of projects',
            'Any Change in RTL?_cum_changed':'No of Projects with RTL change',
            'Any Change in RTL?_cum_project_added':'Number of projects Added',
            'Any Change in Project Cost?_cum_changed':'No of Projects with Project Cost Change',
              }

df_draft_pb_summary.rename(dict_rename, axis = 1, inplace=True)

In [38]:
# df_draft_pb_summary

In [39]:
df_summary_combined=pd.merge(df_draft_pb_summary,df_approved_pb_summary, 
                             how = 'left', 
                             left_on = ['District','Project Status'],
                            right_on = ['District','Project Status'])

In [40]:
sub_total = df_summary_combined.groupby(['District']).agg(sum).reset_index()

sub_total['Project Status'] = 'Total'

In [41]:
df_summary_combined = pd.concat([df_summary_combined, sub_total]).sort_values(['District','Project Status'])

In [42]:

ck_col = '% with RTL change'  
df_summary_combined[ck_col] = df_summary_combined['No of Projects with RTL change']/df_summary_combined['Current Draft Book Number of projects']

In [43]:
ck_col = '% with Project Cost Change'  
df_summary_combined[ck_col] = df_summary_combined['No of Projects with Project Cost Change']/df_summary_combined['Current Draft Book Number of projects']

In [44]:
ck_col = 'Total Change in Cost ($K)'  
df_summary_combined[ck_col] = df_summary_combined['Current Draft Book Total Project Cost ($K)']-df_summary_combined['Latest Certified Book Total Project Cost ($K)']

In [45]:
ck_col = 'Total cost change (%)'  
df_summary_combined[ck_col] = df_summary_combined['Total Change in Cost ($K)']/df_summary_combined['Latest Certified Book Total Project Cost ($K)']

In [46]:
df_summary_output = df_summary_combined[[
    'District', 'Project Status',
    'Latest Certified Book Number of projects','Latest Certified Book Total Project Cost ($K)', 
    'Current Draft Book Number of projects', 'Current Draft Book Total Project Cost ($K)',
     'Number of projects Added', 'Number of projects Deleted',
       '% with RTL change',
       '% with Project Cost Change', 
    'Total Change in Cost ($K)',
       'Total cost change (%)']]

In [47]:
# df_summary_output

,District,Project Status,Latest Certified Book Number of projects,Latest Certified Book Total Project Cost ($K),Current Draft Book Number of projects,Current Draft Book Total Project Cost ($K),Number of projects Added,Number of projects Deleted,% with RTL change,% with Project Cost Change,Total Change in Cost ($K),Total cost change (%)
0,1,Planning,43,1079931.00,47,1092815.00,6,0,0.021277,0.085106,12884.00,0.011930
1,1,Post-Planning,77,1567457.00,79,1583847.00,0,0,0.000000,0.012658,16390.00,0.010456
0,1,Total,120,2647388.00,126,2676662.00,6,0,0.007937,0.039683,29274.00,0.011058
2,2,Planning,109,1989854.40,100,1945452.40,0,1,0.000000,0.000000,-44402.00,-0.022314
3,2,Post-Planning,38,939393.00,46,964565.00,0,0,0.000000,0.043478,25172.00,0.026796
1,2,Total,147,2929247.40,146,2910017.40,0,1,0.000000,0.013699,-19230.00,-0.006565
4,3,Planning,82,2121894.00,84,2122064.00,4,0,0.000000,0.023810,170.00,0.000080
5,3,Post-Planning,45,1581389.00,47,1592273.00,0,0,0.085106,0.106383,10884.00,0.006883
2,3,Total,127,3703283.00,131,3714337.00,4,0,0.030534,0.053435,11054.00,0.002985
6,4,Planning,268,5035404.48,264,4967548.08,7,0,0.000000,0.007576,-67856.40,-0.013476


In [48]:
df_summary_output['Data_TimeStamp'] = Data_TimeStamp
uf.export_hyper(df_summary_output, 'Draft_ProjectBook_Change_Summary', LOG_FILE)

<ipython-input-48-40b696403123>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summary_output['Data_TimeStamp'] = Data_TimeStamp
processing table: 36it [00:00, 12030.51it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing Draft_ProjectBook_Change_Summary.hyper to Sandbox_ProjectBookCheck_Automation...


In [49]:
# ck_col = 'Is Project in the 2022 PID workplan (No Reservation Project) or a Long Lead that will be a candidate for fully programmed into the 2022 SHOPP?'

# #Excel formula
# # =IF(OR(H3="Safety Improvements",H3="Major Damage - Permanent Restoration"),"No",
# # IF(AND(I3="Planning",OR(RIGHT(J3,2)*1=25,RIGHT(J3,2)*1=26)),"Yes",
# # IF(AND(I3="Planning",AA3="Yes",AB3="2022/23"),"Yes",
# # IF(AND(I3="Post-Planning",AA3="Yes",OR(RIGHT(J3,2)*1=25,RIGHT(J3,2)*1=26)),"Yes",
# # "No"))))

# def ck_2022_SHOPP_candidate(df):
#     if df['Advertised Year'] == 'Added to Project Book':
#         return 'Added to Project Book'
#     if df['Activity'] in ['Safety Improvements', 'Major Damage - Permanent Restoration']:
#         return 'No'
#     elif df['Planning or Post-Planning'] == 'Planning':
#         if int(df['Advertised Year'][-2:]) in [25, 26]:
#             return 'Yes'
#         elif df['Active Long Lead'] == "Yes":
#             return 'Yes'
#         else: 
#             return 'No'
#     elif df['Planning or Post-Planning'] == 'Post-Planning':
#         if df['Active Long Lead'] == "Yes" and int(df['Advertised Year'][-2:]) in [25, 26]:
#             return 'Yes'
#         else:
#             return 'No' 
#     else: #this is redundant
#         return 'No'

    
# df_draft_pb_combined[ck_col] = df_combined_pb.apply(ck_2022_SHOPP_candidate, axis=1)

## Raw Data Bridge Worksheet


In [50]:
#rename columns 
#with manual edits

dict_rename_bridge_worksheet = {
 'ID': 'AMT_ID',
 'Bridge №': 'BridgeNo',
 'Work Type': 'WorkType',
 'Brdige / TunnelWork Description': 'WorkDescription',
 'Bridge /TunnelHealth Pre': 'Health Pre',
 'Bridge /TunnelHealth Post': 'Health Post',
 'BridgeScourPre': 'Scour_Pre',
 'BridgeScourPost': 'Scour_Post',
 'BridgeSeismicPre': 'Seismic_Pre',
 'BridgeSeismicPost': 'Seismic_Post',
 'BridgeGds MvmtPre': 'GdsMvmt_Pre',
 'BridgeGds MvmtPost': 'GdsMvmt_Post',
 'Exist(sf)': 'Deck_Exist(sf)',
 'Additional(sf)': 'Deck_Additional(sf)',
 'Y/N': 'Paint_Y/N',
 'Condition': 'Paint_Condition',
 'Paint Area(sf)': 'Paint Area(sf)',
 'Y/N.1': 'ElectricalMechanical_Y/N',
 'Condition.1': 'ElectricalMechanical_Condition',
 'Area(sf)': 'ElectricalMechanical_Area(sf)',
 'Y/N.2': 'ApproachSlab_Y/N',
 'Replaced(sf)': 'ApproachSlab_Replaced(sf)',
 'New(sf)': 'ApproachSlab_New(sf)',
 'Y/N.3': 'Rail_Y/N',
 'Good(lf)': 'Rail_Good(lf)',
 'Fair(lf)': 'Rail_Fair(lf)',
 'Poor(lf)': 'Rail_Poor(lf)',
 'Additonal(lf)': 'Rail_Additonal(lf)',
 'Post Good(lf)': 'Rail_Post Good(lf)',
 'Post Fair(lf)': 'Rail_Post Fair(lf)',
 'Post Poor(lf)': 'Rail_Post Poor(lf)',
 'Post New(lf)': 'Rail_Post New(lf)',
 'FishPassage(Y/N)': 'FishPassage(Y/N)',
}

df_brg_raw_data.rename(dict_rename_bridge_worksheet, axis = 1, inplace = True)

In [51]:
df_brg_raw_data.name = 'df_brg_raw_data'

In [52]:
df_brg_raw_data['Rail_Good(lf)'].fillna(0, inplace = True)
df_brg_raw_data['Rail_Fair(lf)'].fillna(0, inplace = True)
df_brg_raw_data['Rail_Poor(lf)'].fillna(0, inplace = True)

In [53]:
df_brg_raw_data['Rail_Total(lf)'] = (df_brg_raw_data['Rail_Good(lf)'] 
                                             + df_brg_raw_data[ 'Rail_Fair(lf)'] 
                                             + df_brg_raw_data['Rail_Poor(lf)'])

## Raw Data Drainage Worksheet


In [54]:
df_drain_raw_data.name = 'df_drain_raw_data'

dict_drain_rename = {
 'ID': 'AMT_ID',
 'Data Date':'Data Date_Drainage'
                               }
df_drain_raw_data.rename(dict_drain_rename, axis = 1, inplace = True)

cols = ['EA','EFIS','SYSNO','INETNO','OUTETNO']
for c in cols:
    df_drain_raw_data[c] = df_drain_raw_data[c].apply(uf.remove_punction)


df_drain_raw_data['Data Date_Drainage'] = df_drain_raw_data['Data Date_Drainage'].apply(uf.regulate_timestamp_format)

In [55]:

df_drain_raw_data['Unique Culvert ID'] = df_drain_raw_data.apply(uf.calc_drain_unique_ID, axis = 1)

In [56]:
# df_drain_raw_data[df_drain_raw_data['AMT_ID']==22780]

In [57]:
df_drain_raw_data[df_drain_raw_data['AMT_ID']==22780]['Is the proposed treatment expected to remediate the fish passage priority barrier? (Yes/No/NA)'].unique()

array([nan, 'No', 'N A', 'Yes'], dtype=object)

## Raw Data TMS Worksheet

In [58]:
dict_TMS_rename = {
 'ID': 'AMT_ID',
 'Data Date':'Data Date_TMS'
                               }
df_tms_raw_data.rename(dict_TMS_rename, axis = 1, inplace = True)

df_tms_raw_data.name = 'df_tms_raw_data'

df_tms_raw_data['Data Date_TMS'] = df_tms_raw_data['Data Date_TMS'].apply(uf.regulate_timestamp_format)

## Raw Data FP Worksheet

In [59]:
#rename columns
dict_rename = {
    'ID': 'AMT_ID',
    'Fish Passage Type(Priority List /Not Priority List)' : 'Fish Passage Type (Priority List / Not Priority List)',
    'Should countas addressingFish Passage(Yes/No)?': 'Should count as addressing Fish Passage (Yes/No)?'
              }

df_fp_raw_data = df_fp_raw_data.rename(dict_rename, axis = 1)

<a id='TenYrShopp_Perf_RawData'></a>
## TenYrShopp_Perf_RawData


In [60]:
#rename columns
dict_rename_perf_rawdata = {
    'ID': 'AMT_ID',
              }
df_perf_raw_data = df_perf_raw_data.rename(dict_rename_perf_rawdata, axis = 1)

In [61]:
cols_strip = ['EA','EFIS','PPNO']
for c in cols_strip :
    df_perf_raw_data[c] = df_perf_raw_data[c].str.strip("'")

In [62]:
#data clean 
#data type regulation

df_perf_raw_data['Quantity'] = df_perf_raw_data['Quantity'].fillna(0)
df_perf_raw_data['Assets in Good Cond'] = df_perf_raw_data['Assets in Good Cond'].fillna(0)
df_perf_raw_data['Assets in Fair Cond'] = df_perf_raw_data['Assets in Fair Cond'].fillna(0)
df_perf_raw_data['Assets in Poor Cond'] = df_perf_raw_data['Assets in Poor Cond'].fillna(0)
df_perf_raw_data['New Assets Added'] = df_perf_raw_data['New Assets Added'].fillna(0)

df_perf_raw_data['EFIS'] = pd.to_numeric(df_perf_raw_data['EFIS'], errors='coerce')


In [63]:
#data trimming
#row
df_perf_raw_data= df_perf_raw_data[df_perf_raw_data['District'] != 56]
#column
df_perf_raw_data.drop(['PID Cycle', 'TYP','ProjectedSHOPP Cycle','RequestedRTL FY','DistrictPriority'],
  axis='columns', inplace=True, errors='ignore')

In [64]:
df_perf_raw_data.name = 'df_perf_raw_data'

# Add columns to performance data

In [65]:
#include only the project in projectbook and active section

df_perf_raw_data_1 = pd.merge(df_perf_raw_data,
                              df_draft_pb[['AMT_ID', 'Section','Planning or Post-Planning', 'Advertised Year', 'New SHOPP Candidate?']], 
                how ='inner', 
                left_on = ['AMT_ID', 'Section'],
                right_on = ['AMT_ID', 'Section'], 
                             )

In [66]:
df_perf_raw_data_1['Post-Fair'].fillna(0, inplace = True)

df_perf_raw_data_1['F2G Achieved'] = df_perf_raw_data_1['Post-Fair'] - df_perf_raw_data_1['Assets in Fair Cond']

In [67]:
df_perf_raw_data_1['Post-Poor'].fillna(0, inplace = True)

df_perf_raw_data_1['P2G Achieved'] = df_perf_raw_data_1['Post-Poor'] - df_perf_raw_data_1['Assets in Poor Cond']

In [68]:
dict_rename_performance ={
    'New Assets Added':'New Achieved',
}

df_perf_raw_data_1 = df_perf_raw_data_1.rename(dict_rename_performance, axis = 1)


In [69]:
df_perf_raw_data_1['Performance Objective'].fillna('',inplace = True)

In [70]:
df_perf_raw_data_1['Concatenate ID+Objective'] = df_perf_raw_data_1['AMT_ID'].astype(str) + df_perf_raw_data_1['Performance Objective']

In [71]:
def ck_reviewed(df):
    if pd.isna(df['Review Date']):
        return 'No'
    else:
        return 'Yes'
        
df_perf_raw_data_1['Reviewed?'] = df_perf_raw_data_1.apply(ck_reviewed, axis = 1)   

In [72]:
# df_perf_raw_data_1.shape

## common functions

In [73]:
def calc_review_status(df, col_name, NA_msg = 'No relavent data for review'):
    
    # if there is active entry of "NO" or "New", no active entry of "YES" --> Needs Review
    # if there is active entry of "NO" or "New", and active entry of "YES" --> Partially Reviewed
    # if there is no active entry of "NO" or "New", and active entry of "YES" --> Review Complete
    # if there is no active entry of "NO" or "New", no active entry of "YES" --> No relavent data for review
        
    if ('No' in df[col_name]) : 
        if 'Yes' in df[col_name]:
            return 'Partially Reviewed'
        else:
            return 'Needs Review'
    elif 'Yes' in df[col_name]:
        return 'Review Complete'
    elif 'New' in df[col_name]:
        return 'All New'
    else:
        return NA_msg
        

In [74]:
#keep only projects within project book and active section

df_brg_raw_data_1 = pd.merge(df_brg_raw_data, df_draft_pb[['AMT_ID', 'Section']], 
                             how = 'inner', left_on =['AMT_ID', 'Section'],right_on =['AMT_ID', 'Section'],)

#keep only projects within project book and active section

df_tms_raw_data_1 = pd.merge(df_tms_raw_data, df_draft_pb[['AMT_ID', 'Section']], 
                             how = 'inner', left_on =['AMT_ID', 'Section'],right_on =['AMT_ID', 'Section'],)


#keep only projects within project book and active section

df_drain_raw_data_1 = pd.merge(df_drain_raw_data, df_draft_pb[['AMT_ID', 'Section']], 
                             how = 'inner', left_on =['AMT_ID', 'Section'],right_on =['AMT_ID', 'Section'],)


df_fp_raw_data_1= pd.merge(df_fp_raw_data, df_draft_pb[['AMT_ID', 'Section']], 
                             how = 'inner', left_on =['AMT_ID', 'Section'],right_on =['AMT_ID', 'Section'],)



In [75]:
# add Bridge WS Health Review Status column to performance raw data
def ck_brg_health_data(df):
    if pd.isna(df['Health Pre']):
        return "No Bridge Health"
    elif pd.isna(df['Health Post']):
        return 'No'
    else:
        return 'Yes'
        
df_brg_raw_data_1['Post-Condition for Bridge Health entered?']= df_brg_raw_data_1.apply(ck_brg_health_data, axis = 1)     

temp1 = df_brg_raw_data_1.groupby(['AMT_ID', 'Section'])['Post-Condition for Bridge Health entered?'].agg(['unique']).reset_index()

temp1['Bridge WS Health Review Status'] = temp1.apply(calc_review_status, args = ['unique'], axis = 1)
        
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section','Bridge WS Health Review Status']], 
                              how = 'left', left_on = ['AMT_ID', 'Section'],  right_on = ['AMT_ID', 'Section'])

df_perf_raw_data_1['Bridge WS Health Review Status'].fillna('No Bridge Worksheet', inplace = True)

In [76]:
# df_perf_raw_data_1.groupby(['AMT_ID', 'Section'])['Post-Condition for Bridge Health entered?'].agg(['unique']).reset_index()


In [77]:
# df_perf_raw_data_1.shape

In [78]:
# temp1 = df_brg_raw_data_1.groupby(['AMT_ID', 'Section'])['Post-Condition for Bridge Health entered?'].agg(['unique']).reset_index()

# temp1['Bridge WS Health Review Status'] = temp1.apply(calc_review_status, args = ['unique'], axis = 1)


# temp1[temp1['AMT_ID'] ==  11281]


In [79]:
# add TMS WS Health Review Status column to performance raw data

def ck_tms_data(df):
    if pd.isna(df['Asset Post-Condition']):
        return 'No'
    elif df['Asset Post-Condition'] == 'New':
        return 'New'
    else:
        return 'Yes'
        
df_tms_raw_data_1['Post-Condition entered?']= df_tms_raw_data_1.apply(ck_tms_data, axis = 1)     

temp1 = df_tms_raw_data_1.groupby(['AMT_ID', 'Section','TMS Structural or Technology'])['Post-Condition entered?'].agg(['unique']).reset_index()

temp1['TMS WS Review Status'] = temp1.apply(calc_review_status, args = ['unique', 'No TMS Worksheet'], axis = 1)


In [80]:
temp1['Performance Objective'] = temp1.apply(lambda df: 'Transportation Management System Structures'  if 'Structures' in df['TMS Structural or Technology'] else np.nan, axis = 1)
temp1['TMS Structure Review Status'] = temp1.apply(lambda df: df['TMS WS Review Status']  if 'Structures' in df['TMS Structural or Technology'] else np.nan, axis = 1)
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section','Performance Objective','TMS Structure Review Status']], 
                              how = 'left', left_on = ['AMT_ID', 'Section','Performance Objective'],  right_on = ['AMT_ID', 'Section','Performance Objective'])

In [81]:
temp1['Performance Objective'] = temp1.apply(lambda df: 'Transportation Management Systems' if 'Technology' in df['TMS Structural or Technology'] else np.nan, axis = 1)
temp1['TMS Technology Review Status'] = temp1.apply(lambda df: df['TMS WS Review Status']  if 'Technology' in df['TMS Structural or Technology'] else np.nan, axis = 1)
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section','Performance Objective','TMS Technology Review Status']], 
                              how = 'left', left_on = ['AMT_ID', 'Section','Performance Objective'],  right_on = ['AMT_ID', 'Section','Performance Objective'])

In [82]:
def combine_tms_ws_reviews(df):
    '''
    combine review status of the two columns of the tms structure and tms technology
    '''
    if pd.isnull(df['TMS Technology Review Status']):
        return df['TMS Structure Review Status']
    else:
        return df['TMS Technology Review Status']
df_perf_raw_data_1['TMS WS Review Status'] = df_perf_raw_data_1.apply(combine_tms_ws_reviews, axis = 1)

In [83]:
# df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section','TMS Structural or Technology','TMS WS Review Status']], 
#                               how = 'left', left_on = ['AMT_ID', 'Section'],  right_on = ['AMT_ID', 'Section'])

df_perf_raw_data_1['TMS WS Review Status'].fillna('No TMS Worksheet', inplace = True)

In [84]:
# df_tms_raw_data_1.head()

In [85]:
# df_perf_raw_data_1['Performance Objective'].unique()

In [86]:
# AMT_ID = 19289
# df_perf_raw_data_1[(df_perf_raw_data_1['AMT_ID'] == AMT_ID)
#                    & (df_perf_raw_data_1['Performance Objective'].isin(['Transportation Management System', 'Transportation Management System Structures']))
# #                   & (df_perf_raw_data_1['TMS Structural or Technology'] == 'Technology')
                   
# #                    & (df_perf_raw_data_1['TMS Structural or Technology'] == 'Technology & Structures')
#                   ]

In [87]:
# temp1 = df_tms_raw_data_1.groupby(['AMT_ID', 'Section','TMS Structural or Technology'])['Post-Condition entered?'].agg(['unique']).reset_index()

# temp1['TMS WS Review Status'] = temp1.apply(calc_review_status, args = ['unique'], axis = 1)

# temp1[temp1['AMT_ID'] == AMT_ID]

In [88]:
# df_tms_raw_data_1[df_tms_raw_data_1['AMT_ID'] == AMT_ID]

In [89]:
def combine_review_status_for_same_performance_objective(df, ws_review_status_col, WS_missing_msg):
    
    if WS_missing_msg in df[ws_review_status_col] :
        return WS_missing_msg
    
    elif len(df['Reviewed?']) ==1 and ('No' in df['Reviewed?']):
        if 'Review Complete' not in df[ws_review_status_col]:
            return 'Needs Review'
        else: 
            return 'Partially Reviewed'
    else:    # df['Reviewed?'] = 'Yes'
        if 'Needs Review' not in df[ws_review_status_col]:
            return 'Review Complete'
        else:
            return 'Partially Reviewed'

In [90]:
# df_perf_raw_data_2 = df_perf_raw_data_1.copy()

In [91]:
# df_perf_raw_data_1 = df_perf_raw_data_2

In [92]:
performance_objective = 'Transportation Management Systems'
ws_review_status_col = 'TMS WS Review Status'
combined_ws_review_status_col = 'Combined TMS Technology Review Status'

temp1 = df_perf_raw_data_1[
    (df_perf_raw_data_1['Performance Objective'] == performance_objective) 
#     & (df_perf_raw_data_1['TMS Structural or Technology'] == 'Technology')                 
                          ].groupby(['AMT_ID', 'Section','Performance Objective'])['Reviewed?', ws_review_status_col ].agg(set).reset_index()

temp1[combined_ws_review_status_col] = temp1.apply(
    combine_review_status_for_same_performance_objective, 
    args = [ws_review_status_col,'No TMS Worksheet'], axis = 1)


df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section', 'Performance Objective',combined_ws_review_status_col]], 
                              how = 'left', left_on = ['AMT_ID', 'Section','Performance Objective'],  right_on = ['AMT_ID', 'Section','Performance Objective'])

<ipython-input-92-73290855e2b7>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp1 = df_perf_raw_data_1[


In [93]:
performance_objective = 'Transportation Management System Structures'
ws_review_status_col = 'TMS WS Review Status'
combined_ws_review_status_col = 'Combined TMS Structures Review Status'

temp1 = df_perf_raw_data_1[
    (df_perf_raw_data_1['Performance Objective'] == performance_objective)
#     &(df_perf_raw_data_1['TMS Structural or Technology'] == 'Technology & Structures')        
                          ].groupby(['AMT_ID', 'Section','Performance Objective'])['Reviewed?', ws_review_status_col ].agg(set).reset_index()

temp1[combined_ws_review_status_col] = temp1.apply(
    combine_review_status_for_same_performance_objective, 
    args = [ws_review_status_col,'No TMS Worksheet'], axis = 1)


df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section', 'Performance Objective',combined_ws_review_status_col]], 
                              how = 'left', left_on = ['AMT_ID', 'Section','Performance Objective'],  right_on = ['AMT_ID', 'Section','Performance Objective'])

<ipython-input-93-85585e497dfa>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp1 = df_perf_raw_data_1[


In [94]:
performance_objective = 'Bridge and Tunnel Health'
ws_review_status_col = 'Bridge WS Health Review Status'
combined_ws_review_status_col = 'Combined Bridge Health Review Status'

temp1 = df_perf_raw_data_1[df_perf_raw_data_1['Performance Objective'] == performance_objective].groupby(['AMT_ID', 'Section', 'Performance Objective'])['Reviewed?', ws_review_status_col ].agg(set).reset_index()

temp1[combined_ws_review_status_col] = temp1.apply(
    combine_review_status_for_same_performance_objective, 
    args = [ws_review_status_col, 'No Bridge Worksheet'], axis = 1)


df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section', combined_ws_review_status_col]], 
                              how = 'left', left_on = ['AMT_ID', 'Section'],  right_on = ['AMT_ID', 'Section'])

<ipython-input-94-44914227c22e>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp1 = df_perf_raw_data_1[df_perf_raw_data_1['Performance Objective'] == performance_objective].groupby(['AMT_ID', 'Section', 'Performance Objective'])['Reviewed?', ws_review_status_col ].agg(set).reset_index()


In [95]:
#group performance objectives and calculate review status for groupable ACT Ids
#summarize review status for each group of [AMT_ID, Section, Combined ActID]

def group_target_act_id(df):
    '''
    CS, SLR, ADA, 
    '''
    if df['ActID'] in ['H05','H06','H08','H13','H21','H33']:
        return 'Complete Street'
    elif df['ActID'] in ['I19','I20']:
        return 'Sea Level Rise'
    elif df['ActID'] in ['F21','F22','F23','F24','F25','F26','F27','F28', 'F31', 'F34']:
        return 'ADA'    
    else:
        return 'Can not be combined'
    
df_perf_raw_data_1['Combined ActID'] = df_perf_raw_data_1.apply(group_target_act_id, axis = 1)     

temp1 = df_perf_raw_data_1.groupby(['AMT_ID', 'Section','Combined ActID'])['Reviewed?'].agg(['unique']).reset_index()

temp1['Combined ActID Review Status'] = temp1.apply(calc_review_status, args = ['unique'], axis = 1)   

temp1[temp1['Combined ActID']!= 'Can not be combined']['Combined ActID Review Status'] = 'Not Applicable'

<ipython-input-95-b6a7f2088214>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp1[temp1['Combined ActID']!= 'Can not be combined']['Combined ActID Review Status'] = 'Not Applicable'


In [96]:
rename_dict = {
    'Combined ActID': 'Combined Performance Objective',
}

temp1 = temp1.rename(rename_dict, axis = 1)

In [97]:
def mark_combined_act_id(df):
    if df['Performance Objective'] in ["Complete Streets Fix Existing","Complete Streets Build New"]:
        return 'Complete Street'
        
    elif df['Performance Objective'] in ["ADA Pedestrian Infrastructure"]:
        return 'ADA'

    elif df['Performance Objective'] in ["Sea Level Rise"]:
        return 'Sea Level Rise'
    else:
        return 'Can not be combined'

    
df_perf_raw_data_1['Combined Performance Objective'] = df_perf_raw_data_1.apply(mark_combined_act_id, axis = 1)

In [98]:
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp1[['AMT_ID', 'Section', 'Combined Performance Objective','Combined ActID Review Status']], 
                              how = 'left', left_on = ['AMT_ID', 'Section', 'Combined Performance Objective',],  
                              right_on = ['AMT_ID', 'Section', 'Combined Performance Objective',])

In [99]:
df_perf_raw_data_1 ['Combined ActID Review Status'].fillna('No Valid Act ID Data Available', inplace = True)

In [100]:
#summarize review status for each group of [AMT_ID, Section, Program Objective]

temp = df_perf_raw_data_1.groupby(['AMT_ID', 'Section','Performance Objective'])['Reviewed?'].agg(['unique']).reset_index()

temp['Performance Objective Review Status'] = temp.apply(calc_review_status, args = ['unique'], axis = 1)   

df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, temp[['AMT_ID', 'Section', 'Performance Objective','Performance Objective Review Status']], 
                              how = 'left', left_on = ['AMT_ID', 'Section', 'Performance Objective'],  
                              right_on = ['AMT_ID', 'Section', 'Performance Objective'])

In [101]:
def combine_review_status(df):
    if df['Combined Performance Objective'] == 'Can not be combined':
        return df['Performance Objective Review Status']
    else:
        return df['Combined ActID Review Status']

df_perf_raw_data_1['Review Status'] = df_perf_raw_data_1.apply(combine_review_status, axis = 1)

In [102]:
# df_perf_raw_data_1['Review Status'].value_counts()

## Fish Passage review status

In [103]:
def ck_priority_fp_review(df, FP1, FP2):
    '''
    check if the FP1 column to see if the activity for current data row is within FP priority list
    return 'Not in Priority list' if not in the priority list
    If it is in the priority list, check FP2 column, to see the data should be counted as priority list 
    '''
    if df[FP1] != 'Priority List':
        return "Not in Priority list"
    elif pd.isna(df[FP2]) : 
        return 'No'
    else:
        return 'Yes'

def ck_NONpriority_fp_review(df, FP1, FP2):
    if df[FP1] == 'Priority List':
        return "In Priority list"
    elif pd.isna(df[FP2]): 
        return 'No'
    else:
        return 'Yes'

In [104]:
rename_dict = {
    'Fish PassagePriority List(Yes/No)': 'Fish Passage Priority List (Yes/No)',
    'Is the proposedtreatmentexpected toremediate thefish passagepriority barrier?(Yes/No/NA)': 'Should Count toward Fish Passage Priority List (Yes/No)',
    'PriorityIdentifier': 'Priority Identifier',
    'AddressingFish Passagenot in the Priority List(Yes/No)': 'Addressing Fish Passage not in the Priority List (Yes/No)?',
}

df_brg_raw_data_1 = df_brg_raw_data_1.rename(rename_dict, axis = 1)


rename_dict = {
    'Is the proposed treatment expected to remediate the fish passage priority barrier? (Yes/No/NA)': 'Should Count toward Fish Passage Priority List (Yes/No)',
}

df_drain_raw_data_1 = df_drain_raw_data_1.rename(rename_dict, axis = 1)



rename_dict = {
    'Fish Passage Type (Priority List / Not Priority List)': 'Fish Passage Priority List (Yes/No)',
    'Should count as addressing Fish Passage (Yes/No)?': 'Should Count toward Fish Passage Priority List (Yes/No)',
    'Priority Identifieror FP Identification': 'Priority Identifier',    
}

df_fp_raw_data_1 = df_fp_raw_data_1.rename(rename_dict, axis = 1)


In [105]:
# df_brg_raw_data_1.columns

In [106]:
FP1 = 'Fish Passage Priority List (Yes/No)'
FP2 = 'Should Count toward Fish Passage Priority List (Yes/No)'

df_brg_raw_data_1['Fish Passage in the Priority List'] = df_brg_raw_data_1['Fish Passage Priority List (Yes/No)'].apply(lambda x: 'Yes' if x == 'Yes' else 'No')
df_brg_raw_data_1['FP in Priority List reviewed by FP program?'] = df_brg_raw_data_1.apply(ck_priority_fp_review, args = [FP1, FP2], axis = 1)  
df_brg_raw_data_1['Fish Passage NOT in the Priority List'] = df_brg_raw_data_1['Addressing Fish Passage not in the Priority List (Yes/No)?'].apply(lambda x: 'Yes' if x == 'Yes' else 'No')
df_brg_raw_data_1['FP NOT in Priority List reviewed by FP program?'] = df_brg_raw_data_1.apply(ck_NONpriority_fp_review, args = [FP1, FP2], axis = 1)     


df_drain_raw_data_1['Fish Passage in the Priority List'] = df_drain_raw_data_1['Fish Passage Priority List (Yes/No)'].apply(lambda x: 'Yes' if x == 'Yes' else 'No')
df_drain_raw_data_1['FP in Priority List reviewed by FP program?'] = df_drain_raw_data_1.apply(ck_priority_fp_review, args = [FP1, FP2], axis = 1)  
df_drain_raw_data_1['Fish Passage NOT in the Priority List'] = df_drain_raw_data_1['Addressing Fish Passage not in the Priority List (Yes/No)?'].apply(lambda x: 'Yes' if x == 'Yes' else 'No')
df_drain_raw_data_1['FP NOT in Priority List reviewed by FP program?'] = df_drain_raw_data_1.apply(ck_NONpriority_fp_review, args = [FP1, FP2], axis = 1)  


df_fp_raw_data_1['Fish Passage in the Priority List'] = df_fp_raw_data_1['Fish Passage Priority List (Yes/No)'].apply(lambda x: 'Yes' if x == 'Priority List' else 'No')
df_fp_raw_data_1['FP in Priority List reviewed by FP program?'] = df_fp_raw_data_1.apply(ck_priority_fp_review, args = [FP1, FP2], axis = 1)
df_fp_raw_data_1['Fish Passage NOT in the Priority List'] = df_fp_raw_data_1['Fish Passage Priority List (Yes/No)'].apply(lambda x: 'No' if x == 'Priority List' else 'Yes')
df_fp_raw_data_1['FP NOT in Priority List reviewed by FP program?'] = df_fp_raw_data_1.apply(ck_NONpriority_fp_review, args = [FP1, FP2], axis = 1)  

In [107]:
# df_drain_raw_data_1[df_drain_raw_data_1['AMT_ID']==22867]

In [108]:
target_cols = ['AMT_ID', 'Section','District','Priority Identifier','Should Count toward Fish Passage Priority List (Yes/No)']

In [109]:
temp = df_brg_raw_data_1[df_brg_raw_data_1['Fish Passage Priority List (Yes/No)'] == 'Yes'][target_cols]
temp['Worksheet'] = 'Bridge'

temp1 = df_drain_raw_data_1[df_drain_raw_data_1['Fish Passage Priority List (Yes/No)'] == 'Yes'][target_cols]
temp1['Worksheet'] = 'Drainage'
temp = temp.append(temp1)

temp2 = df_fp_raw_data_1[df_fp_raw_data_1['Fish Passage Priority List (Yes/No)'] == 'Priority List'][target_cols]
temp2['Worksheet'] = 'Fish Passage'
temp = temp.append(temp2)

In [110]:
temp['Priority Identifier'].fillna(0, inplace = True)
temp['Priority Identifier'] = temp['Priority Identifier'].astype(int)

In [111]:
#bookmark

temp[temp['AMT_ID']==22780]

,AMT_ID,Section,District,Priority Identifier,Should Count toward Fish Passage Priority List (Yes/No),Worksheet
6641,22780,TYP,5,201855,No,Drainage
6642,22780,TYP,5,201855,N A,Drainage
6643,22780,TYP,5,201855,N A,Drainage
6644,22780,TYP,5,201856,N A,Drainage
6645,22780,TYP,5,201856,Yes,Drainage
6646,22780,TYP,5,201856,N A,Drainage


In [112]:
temp['Should Count toward Fish Passage Priority List (Yes/No)'].fillna('Needs Review', inplace = True)

temp_group = temp.groupby(['Worksheet','AMT_ID', 'Section', 'District', 'Priority Identifier'])['Should Count toward Fish Passage Priority List (Yes/No)'].agg('value_counts').reset_index(name = 'Counts')

df_fp_list = temp_group.pivot(index=['Worksheet','AMT_ID', 'Section', 'District', 'Priority Identifier',], columns='Should Count toward Fish Passage Priority List (Yes/No)', values='Counts').reset_index()


In [113]:
temp_group

,Worksheet,AMT_ID,Section,District,Priority Identifier,Should Count toward Fish Passage Priority List (Yes/No),Counts
0,Bridge,11358,TYP,4,201431,Yes,1
1,Bridge,22816,TYP,10,201760,Yes,1
2,Bridge,23138,TYP,1,201604,Yes,1
3,Drainage,20270,TYP,1,201608,Yes,1
4,Drainage,20270,TYP,1,201809,Yes,1
5,Drainage,20275,PRG,1,201601,Yes,1
6,Drainage,20286,TYP,1,201603,Yes,1
7,Drainage,20286,TYP,1,201803,Yes,1
8,Drainage,21634,TYP,2,201421,N A,2
9,Drainage,21634,TYP,2,201421,Yes,1


In [114]:
# df_fp_list

In [115]:
#add columns if the pivot table did not generate this column
if 'Needs Review' not in df_fp_list.columns:
    df_fp_list['Needs Review'] = 0

if 'N A' not in df_fp_list.columns:
    df_fp_list['N A'] = 0

if 'No' not in df_fp_list.columns:
    df_fp_list['No'] = 0

if 'Yes' not in df_fp_list.columns:
    df_fp_list['Yes'] = 0

In [116]:
df_fp_list.fillna(0, inplace = True)

df_fp_list['Grand Total'] = df_fp_list['Needs Review'] + df_fp_list['N A'] + df_fp_list['No'] + df_fp_list['Yes']

In [117]:
df_fp_list = pd.merge(df_fp_list, df_draft_pb[['AMT_ID','Section','EA_','Advertised Year',]],
                     how = 'left', left_on = ['AMT_ID','Section',], right_on = ['AMT_ID','Section',]
                     )

In [118]:
df_fp_list['RTL'] =  df_fp_list['Advertised Year'].apply(lambda x: int(x[-2:]) +2000)

In [119]:
rename_dict = {
    'EA_': 'EA',
    'Needs Review': 'Should Count Towards FP Priority List: Needs Review',
    'N A':'Should Count Towards FP Priority List: Not Applicable',
    'No':'Should Count Towards FP Priority List: No',
    'Yes':'Should Count Towards FP Priority List: Yes',
    'Grand Total': 'Should Count Towards FP Priority List: Grand Total',
}

df_fp_list = df_fp_list.rename(rename_dict, axis = 1)

In [120]:
df_fp_list['EA'] = df_fp_list['EA'].apply(uf.remove_punction)

In [121]:
df_fp_list= df_fp_list[['Worksheet', 'AMT_ID', 'Section', 'District', 'EA','RTL', 
                        'Priority Identifier',
       'Should Count Towards FP Priority List: Needs Review',
        'Should Count Towards FP Priority List: Not Applicable',
       'Should Count Towards FP Priority List: No',
       'Should Count Towards FP Priority List: Yes',
       'Should Count Towards FP Priority List: Grand Total', ]]

In [122]:
# df_fp_list

In [123]:
temp = df_fp_list.groupby(['Worksheet', 'AMT_ID', 'Section','District', 'EA','RTL',])[[
    'Should Count Towards FP Priority List: Needs Review',
    'Should Count Towards FP Priority List: Not Applicable',
       'Should Count Towards FP Priority List: No',
       'Should Count Towards FP Priority List: Yes',
       'Should Count Towards FP Priority List: Grand Total', ]].sum().reset_index()

In [124]:
def calc_FP_review_status(df):
#     if df['Priority Identifier'] == 0: 
#         return 'No priority FP identified'
    
    if df['Should Count Towards FP Priority List: Needs Review'] == 0:
        return 'Review Complete'
    elif df['Should Count Towards FP Priority List: Not Applicable'] + df['Should Count Towards FP Priority List: No'] + df['Should Count Towards FP Priority List: Yes'] == 0:
        return 'Needs Review'
    else:
        return 'Partially Reviewed'
temp['Priority FP Review Status'] = temp.apply(calc_FP_review_status, axis = 1)

In [125]:
def calc_fp_worksheet(df):
    if df['Performance Objective'] == 'Fish Passage':
        if df['Perf Activity Category'] == 'Sustainability/Climate Change':
            return 'Fish Passage'
        else:
            return df['Perf Activity Category']
    else:
        return 'NA'
    
df_perf_raw_data_1['Worksheet'] = df_perf_raw_data_1.apply(calc_fp_worksheet, axis = 1)

In [126]:
df_perf_raw_data_1 = pd.merge(df_perf_raw_data_1, 
                              temp[['AMT_ID','Section','Worksheet','Priority FP Review Status']], 
                              how = 'left', 
                              left_on = ['AMT_ID','Section','Worksheet'], 
                              right_on = ['AMT_ID','Section','Worksheet'])

In [127]:
# df_fp_list.groupby(['AMT_ID','Section','Worksheet'])['AMT_ID'].count()

In [128]:
df_perf_raw_data_1['Priority FP Review Status'].fillna('No priority FP identified', inplace = True)

In [129]:
def combine_review_status_columns(df):
    '''
    combine review status from different review status columns into one combined review status column
    '''
    AMT_ID = df['AMT_ID']
    Section = df['Section']
    
    if df['Performance Objective'] == 'Fish Passage':
        return df['Priority FP Review Status']
    
    elif df['Performance Objective'] == "Bridge and Tunnel Health":
        return df['Combined Bridge Health Review Status']
    
    elif df['Performance Objective'] == "Transportation Management Systems":
        return df['Combined TMS Technology Review Status']
    
    elif df['Performance Objective'] == "Transportation Management System Structures":
        return df['Combined TMS Structures Review Status']        
    
    else:
        return df['Review Status'] 
    
df_perf_raw_data_1['Combined Review Status'] = df_perf_raw_data_1.apply(combine_review_status_columns, axis = 1)   

In [130]:
# AMT_ID = 18672 
# Section = 'PRG'

# df_perf_raw_data_1[(df_perf_raw_data_1['AMT_ID'] == AMT_ID) & (df_perf_raw_data_1['Section'] == Section)
#                    & (df_perf_raw_data_1['Performance Objective'] == 'Complete Streets Fix Existing')
#                 ]

In [131]:
# AMT_ID = 18672 
# Section = 'PRG'

# df_perf_raw_data_1[(df_perf_raw_data_1['AMT_ID'] == AMT_ID) & (df_perf_raw_data_1['Section'] == Section)
#                    & (df_perf_raw_data_1['Combined ActID'] == "Complete Street")
#                 ]

In [132]:
def ck_HQ_review_complete(df):
    # if modified after review, return Needs Review
    if pd.notna(df['PerformanceChange Date After Review']):
        return 'Needs Re-review'
    else: #
        return df['Combined Review Status']
     
df_perf_raw_data_1['Is HQ Review Complete?'] = df_perf_raw_data_1.apply(ck_HQ_review_complete, axis = 1)   

df_perf_raw_data_1['Is HQ Review Complete?'].fillna('No Need for Review', inplace = True)

In [133]:
df_perf_raw_data_1['PerformanceChange Date After Review'].fillna('NA', inplace = True)

In [134]:
# #DEBUG: Bridge Health
# # AMT_ID = 13550
# AMT_ID = 23253
# AMT_ID = 21974

# df_perf_raw_data_1[
#     (df_perf_raw_data_1['AMT_ID'] == AMT_ID)
#     & (df_perf_raw_data_1['Performance Objective'].isin(["Bridge and Tunnel Health"]))
# #     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
# ][['AMT_ID','Section','ActID','Review Date','Reviewed?','Worksheet','Combined ActID','Bridge WS Health Review Status','Is HQ Review Complete?']]

In [135]:
# df_brg_raw_data_1[df_brg_raw_data_1['AMT_ID'] == AMT_ID][['AMT_ID', 'Section','Health Pre','Health Post','Post-Condition for Bridge Health entered?']]

In [136]:
# #DEBUG: TMS Technology

# AMT_ID = 15955   # No TMS worksheet
# AMT_ID = 21663   # Partially Reviewed
# # AMT_ID = 19289  # Needs Review
# AMT_ID = 19939   # Partially Reviewed
# df_perf_raw_data_1[
#     (df_perf_raw_data_1['AMT_ID'] == AMT_ID)
#     & (df_perf_raw_data_1['Performance Objective']=="Transportation Management Systems")
# #     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
# ][['AMT_ID','Section','ActID','Review Date','Reviewed?','Worksheet','Performance Objective','TMS WS Review Status','Combined TMS Technology Review Status','Is HQ Review Complete?']]

In [137]:
# #DEBUG: TMS Structures
# AMT_ID = 19543   # Partially Reviewed

# df_perf_raw_data_1[
#     (df_perf_raw_data_1['AMT_ID'] == AMT_ID)
#     & (df_perf_raw_data_1['Performance Objective'].isin(["Transportation Management System Structures"]))
# #     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
# ][['AMT_ID','Section','ActID','Review Date','Reviewed?','Worksheet','Performance Objective','TMS WS Review Status','Combined TMS Technology Review Status','Combined TMS Structures Review Status','Is HQ Review Complete?']]

In [138]:
# df_tms_raw_data_1[(df_tms_raw_data_1['AMT_ID'] == AMT_ID)]
# [['AMT_ID','Section','TMS Structural or Technology','Post-Condition entered?']]

In [139]:
# #DEBUG: Complete Street

# AMT_ID = 20245
# df_perf_raw_data_1[
#     (df_perf_raw_data_1['AMT_ID'] == AMT_ID)
# #     & (df_perf_raw_data_1['Performance Objective'].isin(["Complete Streets Fix Existing","Complete Streets Build New"]))
# #     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
# ][['AMT_ID','Section','ActID','Review Date','Reviewed?','Worksheet',
#    'Combined ActID','Combined ActID Review Status','Performance Objective','Performance Objective Review Status','Review Status','Is HQ Review Complete?']]

In [140]:
#DEBUG Fish Passage

AMT_ID = 22780
# AMT_ID = 20275
df_perf_raw_data_1[
    (df_perf_raw_data_1['AMT_ID'] == AMT_ID)
    & (df_perf_raw_data_1['Performance Objective'] == 'Fish Passage')
#     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
][['AMT_ID','Section','ActID','Review Date','Reviewed?','Worksheet','Combined ActID','Priority FP Review Status']]

,AMT_ID,Section,ActID,Review Date,Reviewed?,Worksheet,Combined ActID,Priority FP Review Status
10223,22780,TYP,C17,NaN,No,Drainage,Can not be combined,Review Complete


In [141]:
# df_fp_list[df_fp_list['AMT_ID'] == AMT_ID]

In [142]:
# df_perf_raw_data_1[
#     (df_perf_raw_data_1['AMT_ID'] == 20240)
#     & (df_perf_raw_data_1['Performance Objective'] == 'Complete Streets Fix Existing')
# #     & (df_perf_raw_data_1['Performance Objective'] == 'No Performance Objective in the SHSMP')
# ]


<a id='Export_Data'></a>

# Export Data


In [143]:
df_drain_raw_data_1['Should Count toward Fish Passage not in the Priority List (Yes/No)'].fillna('N/A', inplace = True)

In [144]:
df_brg_raw_data_1['Data_TimeStamp'] = Data_TimeStamp
df_drain_raw_data_1['Data_TimeStamp'] = Data_TimeStamp
df_fp_raw_data_1['Data_TimeStamp'] = Data_TimeStamp
df_fp_list['Data_TimeStamp'] = Data_TimeStamp


uf.export_csv(df_brg_raw_data_1, 'bridge_worksheet', PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)
uf.export_csv(df_drain_raw_data_1, 'drainage_worksheet', PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)
uf.export_csv(df_fp_raw_data_1, 'fishpassage_worksheet', PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)
uf.export_csv(df_fp_list, 'fishpassage_list', PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)

uf.export_hyper(df_brg_raw_data_1, 'bridge_worksheet', LOG_FILE)
uf.export_hyper(df_drain_raw_data_1, 'drainage_worksheet', LOG_FILE)
uf.export_hyper(df_fp_raw_data_1, 'fishpassage_worksheet', LOG_FILE)
uf.export_hyper(df_fp_list, 'fishpassage_list', LOG_FILE)

processing table: 892it [00:00, 6328.06it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing bridge_worksheet.hyper to Sandbox_ProjectBookCheck_Automation...


processing table: 11251it [00:02, 5504.03it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing drainage_worksheet.hyper to Sandbox_ProjectBookCheck_Automation...


processing table: 5it [00:00, 2500.18it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing fishpassage_worksheet.hyper to Sandbox_ProjectBookCheck_Automation...


processing table: 26it [00:00, 13061.67it/s]


Signing into AssetManagement at https://tableau.dot.ca.gov
Publishing fishpassage_list.hyper to Sandbox_ProjectBookCheck_Automation...


In [145]:
def convert_quantity(df):
    '''
    get the numerical rows of the quanity, assign zero for non-numerical rows
    '''
    try: 
        return float(df['Quantity'])
    except:
        if df['Quantity'] == 'Yes':
            return 1
        else:
            return 0
df_perf_raw_data_1['Quantity_Number'] = df_perf_raw_data_1.apply(convert_quantity, axis = 1)

In [146]:
# export performance raw data with review summary
df_perf_raw_data_1['Data_TimeStamp'] = Data_TimeStamp

out_cols = [
    'District', 'AMT_ID', 'EA', 'EFIS', 'PPNO', 'Location', 'County',
       'Route', 'BackPM', 'AheadPM', 'ProjectedRTL FY',
       'Main Activity Category', 'Section', 'ActID', 'Perf Activity Category',
       'Activity Detail', 'Performance Objective', 'Unit of Measurement',
       'Quantity_Number', 'Assets in Good Cond', 'Assets in Fair Cond',
       'Assets in Poor Cond', 'New Achieved', 'Comment', 'Guidance',
       'Last Saved', 'Saved By', 'Post-Good', 'Post-Fair', 'Post-Poor',
       'HQ ProgramReview - Agree with District?', 'HQ Comment', 'Review Date',
       'PerformanceChange Date After Review', 'Status','Concatenate ID+Objective',
       'Planning or Post-Planning', 'Advertised Year', 'F2G Achieved',
       'P2G Achieved', 
       'Is HQ Review Complete?','Combined ActID','Combined Performance Objective',
    'New SHOPP Candidate?',
    'Data_TimeStamp']

uf.export_csv(df_perf_raw_data_1[out_cols], 'performance_review_summary', PROJECTBOOKCHECK_HTTPSERVER_FOLDER, LOG_FILE)
uf.export_hyper(df_perf_raw_data_1[out_cols], 'performance_review_summary', LOG_FILE)


processing table: 0it [00:00, ?it/s]



<a id='FinalCleanUp'></a>
## Final Clean Up

In [147]:

#clean up tableau publishing log file

import os
import glob
# get a recursive list of file paths that matches pattern
fileList = glob.glob('./*.log')
# Iterate over the list of filepaths & remove each file.
for filePath in fileList:
    try:
        os.remove(filePath)
    except OSError:
        print("Error while deleting file")

In [148]:
end_time =  time.time()
elapsed = end_time - start_time
print('time elapsed : {} seconds'.format(elapsed))

file_export_log = open(LOG_FILE, "a")  # append mode
file_export_log.write('#####time elapsed : {} seconds \n'.format(elapsed))
file_export_log.close()


time elapsed : 73.52468991279602 seconds
